DB連線-資料上傳

In [ ]:
## file
import pandas as pd


df_file = pd.read_excel('./file/20250224-C.xlsx',header = None)
# 確認日期
date = df_file.iat[1,1]
date = date.replace('日期:','').replace('年','/').replace('月','/').replace('日','')
date_list = date.split('/')
year=str(int(date_list[0])+1911)
date = year+'/'+date_list[1]+'/'+date_list[2]

df_file = df_file.iloc[4:,1:]




In [ ]:
import pandas as pd
import pyodbc as sql

server = 'localhost,1433' 
database = 'ReportDb' 
username = 'sa' 
password = 'Password123' 
connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=yes;'
conn = sql.connect(connectionString)

connectionString

In [ ]:

SQL_STATEMENT = """
INSERT INTO ConvertibleBondDaily 
(dataDate, id, name, trade_type, closing_price, change_price, open_price, high_price, low_price,
    trade_count, unit_count, total_amount, avg_price, next_ref_price, next_limit_up, next_limit_down) 
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""




for index, row in df_file.iterrows():
    print(index,": ",sep="")
    if not (pd.isnull(row[1])):
        if (row[1] == "合計"):
            break
        try:
            cursor = conn.cursor()
            cursor.execute(SQL_STATEMENT,
            date,
            row[1],  # id
            row[2] if pd.notnull(row[2]) else None,  # name
            row[3] if pd.notnull(row[3]) else None,  # trade_type
            float(row[4]) if pd.notnull(row[4]) else None,  # closing_price
            float(row[5]) if pd.notnull(row[5]) else None,  # change_price
            float(row[6]) if pd.notnull(row[6]) else None,  # open_price
            float(row[7]) if pd.notnull(row[7]) else None,  # high_price
            float(row[8]) if pd.notnull(row[8]) else None,  # low_price
            int(row[9]) if pd.notnull(row[9]) else None,  # trade_count
            int(row[10]) if pd.notnull(row[10]) else None,  # unit_count
            int(row[11]) if pd.notnull(row[11]) else None,  # total_amount
            float(row[12]) if pd.notnull(row[12]) else None,  # avg_price
            float(row[13]) if pd.notnull(row[13]) else None,  # next_ref_price
            float(row[14]) if pd.notnull(row[14]) else None,  # next_limit_up
            float(row[15]) if pd.notnull(row[15]) else None  # next_limit_down
            )
            print("INSERT SUCCEED")
            conn.commit()
        except Exception as e:
            print(f"Error encountered while processing row {index}")
            print(f"Row data: {row}")
            print(f"Error: {str(e)}")
            break
        

conn.commit()
conn.close()

工作日

In [ ]:
# python 3.x
import datetime
import pandas as pd

start = datetime.datetime.strptime("2021/01/01", "%Y/%m/%d")
end = datetime.datetime.strptime("2030/12/31", "%Y/%m/%d")
date = pd.date_range(start, end)
day = pd.date_range(start, end)
date=date.strftime("%Y/%m/%d")
day=day.strftime("%w")

df_date = pd.DataFrame(date, columns=['Date'])
df_day = pd.DataFrame(day, columns=['DayOfWeek'])

# 使用 pd.concat() 橫向合併
df = pd.concat([df_date, df_day], axis=1)


In [ ]:
import pandas as pd
import pyodbc as sql

server = 'localhost,1433' 
database = 'ReportDb' 
username = 'sa' 
password = 'Password123' 
connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=yes;'
conn = sql.connect(connectionString)

SQL_STATEMENT = """
INSERT INTO Date 
(Date,Day) 
VALUES (?,?)
"""




for index, row in df.iterrows():
    print(index,": ",sep="")
    try:
        cursor = conn.cursor()
        cursor.execute(SQL_STATEMENT,
        row[0],
        row[1])
        print("INSERT SUCCEED")
        conn.commit()
    except Exception as e:
        print(f"Error encountered while processing row {index}")
        print(f"Row data: {row}")
        print(f"Error: {str(e)}")
        break
    

conn.commit()

In [ ]:
import pandas as pd
import pyodbc as sql

server = 'localhost,1433' 
database = 'ReportDb' 
username = 'sa' 
password = 'Password123' 
connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=yes;'
conn = sql.connect(connectionString)
sql_query = """
                    SELECT Top(30) 
                        a.[Date]
                        ,(Case
                            When a.[Day] = '0' Then '(Mon)'
                            When a.[Day] = '1' Then '(Tus)'
                            When a.[Day] = '2' Then '(Wed)'
                            When a.[Day] = '3' Then '(Thu)'
                            When a.[Day] = '4' Then '(Fri)'
                        End) as 'Day'
                        ,ISNULL(b.num,0) as 'Num'
                    FROM [ReportDb].[dbo].[Date] a
                    LEFT JOIN(
                        SELECT 
                        dataDate,
                        COUNT(id) as 'num'
                        From ConvertibleBondDaily
                        WHERE [dataDate] <= GETDATE()
                        GROUP BY dataDate) b on a.[Date] =b.dataDate
                    Where a.[Day] <>  '6' and [Day] <> '5' and [Date] <= GETDATE()
                    ORDER BY [Date] DESC
                    """
        

df = pd.read_sql(sql_query,conn)


list_date = df["Date"].values
list_day = df["Day"].values
list_num = df["Num"].values   
    
result = ""
for i in range(0,len(df)):
    if (df.iloc[i]["Num"]== 0):
        # 狀態符號
        result = result + """
                    <tr>
                        <td>
                          <div class="d-flex px-2 py-1">
                            <div>
                              <span class="badge badge-sm bg-gradient-secondary">Not Exist</span>
                            </div>
                          </div>
                        </td>
                        """
    else:
        result = result+"""
                    <tr>
                        <td>
                          <div class="d-flex px-2 py-1">
                            <div>
                              <span class="badge badge-sm bg-gradient-success">Exist</span>
                            </div>
                          </div>
                        </td>
                        """
    #日期 
    result = result + """
                        <td>
                          <div class="align-middle text-center text-s">
                            <span class="text-xs font-weight-bold">""" + str(df.iloc[i]["Date"]) + """</span>
                          </div>
                        </td>
                        """
    # 星期
    result = result + """
                        <td>
                          <div class="align-middle text-center text-s">
                            <span class="text-xs font-weight-bold">"""+ df.iloc[i]["Day"] + """</span>
                          </div>
                        </td>
                        """
    # 按鈕
    result = result + """
                        <td class="align-middle text-center text-sm">
                          <a class="btn btn-outline-primary btn-sm mb-0" target="_blank" href="">Upload</a>
                        </td>
                        """
    # 數量
    result = result + """
                        <td class="align-middle">
                          <div class="progress-wrapper w-75 mx-auto">
                            <div class="progress-info">
                              <div class="progress-percentage">
                                <span class="text-xs font-weight-bold">"""+ str(df.iloc[i]["Num"]/330*100) +"""%</span>
                              </div>
                            </div>
                            <div class="progress">
                              <div class="progress-bar bg-gradient-info" style="width:"""+ str(df.iloc[i]["Num"]/330*100)  +"""%" role="progressbar" aria-valuenow=" """+ str(df.iloc[i]["Num"])+""""
                                aria-valuemin="0" aria-valuemax="330"></div>
                            </div>
                          </div>
                        </td>
                    </tr>
                        """

print(result)

In [ ]:
import pandas as pd
import pyodbc as sql

server = 'localhost,1433' 
database = 'ReportDb' 
username = 'sa' 
password = 'Password123' 
connectionString = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};Encrypt=yes;TrustServerCertificate=yes;'
try:
    conn = sql.connect(connectionString)
    conn.close()
except e:
    print(e)